In [1]:
!pip3 install bonito-llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 101.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB

In [2]:
import json
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datasets import Dataset, load_dataset
from bonito import Bonito
from vllm import SamplingParams
import random
from typing import List, Dict
import time

INFO 06-08 10:15:38 [__init__.py:243] Automatically detected platform cuda.


In [ ]:
def collect_parliament_data():
    print("🏛️ Collecting Canadian Parliamentary Data...")

    # Sample parliamentary texts with proper structure
    parliament_texts = [
        {
            "text": "Mr. Speaker, I rise today to address the critical issue of economic recovery post-pandemic. Our government has implemented a comprehensive strategy that includes targeted support for small businesses, infrastructure investment totaling $12 billion, and enhanced employment insurance benefits. These measures have resulted in a 3.2% GDP growth and unemployment rates dropping to 5.1%. However, we must acknowledge the challenges ahead, particularly inflation concerns and global supply chain disruptions that continue to affect Canadian families.",
            "speaker": "Hon. Minister of Finance",
            "date": "2024-01-15",
            "topic": "economic_policy"
        },
        {
            "text": "Mr. Speaker, the government's economic policies have failed Canadian families. While they boast about GDP growth, the reality is that cost of living has increased by 8% this year alone. Housing prices remain unaffordable for young Canadians, with average home prices exceeding $700,000 nationally. The government's infrastructure spending has been plagued by delays and cost overruns. We propose immediate action: reduce GST on essential goods, implement rent control measures, and provide direct support to first-time homebuyers through expanded tax credits.",
            "speaker": "Opposition Leader",
            "date": "2024-01-15",
            "topic": "economic_policy"
        },
        {
            "text": "Mr. Speaker, healthcare remains our government's top priority. We have allocated an additional $6 billion to provinces and territories to reduce surgical backlogs created during the pandemic. Our national dental care program has already served over 500,000 Canadians, providing essential services previously unavailable. Mental health funding has been doubled, with new community centers opening across rural and urban areas. The Canada Health Act ensures universal access, and we will defend this principle against any attempts at privatization.",
            "speaker": "Minister of Health",
            "date": "2024-01-20",
            "topic": "healthcare"
        },
        {
            "text": "Mr. Speaker, the healthcare crisis in Canada is undeniable. Emergency room wait times have reached critical levels, with some patients waiting over 12 hours for basic care. The government's funding announcements sound impressive, but implementation has been consistently delayed. Rural communities face doctor shortages, with over 200 communities lacking family physicians. The dental care program, while well-intentioned, has been rolled out poorly with many dentists refusing to participate due to inadequate compensation rates.",
            "speaker": "Health Critic",
            "date": "2024-01-20",
            "topic": "healthcare"
        },
        {
            "text": "Mr. Speaker, Canada's commitment to net-zero emissions by 2050 requires bold action today. Our carbon pricing mechanism has proven effective, reducing emissions while returning money to families through rebates. The Clean Technology Investment Fund has supported over 300 green energy projects, creating 45,000 jobs in the renewable sector. We are investing $9.1 billion in public transit to reduce transportation emissions, which account for 25% of Canada's greenhouse gas emissions. International climate leadership means transitioning away from fossil fuels while supporting affected workers.",
            "speaker": "Minister of Environment",
            "date": "2024-02-01",
            "topic": "environment"
        },
        {
            "text": "Mr. Speaker, the government's environmental record is one of broken promises and missed targets. Despite years of carbon pricing, Canada's emissions have not decreased significantly. The government approved the Trans Mountain pipeline expansion while claiming climate leadership - this contradiction is unacceptable. Small businesses and families bear the burden of carbon taxes while large corporations receive exemptions. We need immediate investment in clean technology manufacturing, support for electric vehicle adoption, and a realistic timeline for phasing out oil and gas subsidies.",
            "speaker": "Environment Critic",
            "date": "2024-02-01",
            "topic": "environment"
        },
        {
            "text": "Mr. Speaker, education is the foundation of Canada's future prosperity. Our government has increased federal transfers to provinces for education by 15%, supporting everything from early childhood education to post-secondary research. The Canada Student Grant program has been expanded, providing up to $6,000 annually for students from low-income families. We are addressing the student debt crisis through interest-free loans and expanded forgiveness programs. Indigenous education funding has been prioritized, with new schools built in 12 First Nations communities this year.",
            "speaker": "Minister of Education",
            "date": "2024-02-10",
            "topic": "education"
        },
        {
            "text": "Mr. Speaker, Canada's immigration system must balance economic needs with humanitarian obligations. Our Immigration Levels Plan targets 465,000 new permanent residents in 2024, focusing on French-speaking immigrants and those with skills in high-demand sectors like healthcare and technology. The Express Entry system has been reformed to prioritize Canadian work experience and official language proficiency. We are also addressing backlogs in refugee processing, with new funding for the Immigration and Refugee Board to reduce wait times from 24 months to 12 months.",
            "speaker": "Immigration Minister",
            "date": "2024-02-15",
            "topic": "immigration"
        },
        {
            "text": "Mr. Speaker, today I address the housing crisis that affects millions of Canadians. Our National Housing Strategy represents the largest federal housing investment in decades, with $72 billion committed over 10 years. We have already created over 100,000 new housing units and repaired 300,000 existing homes. The First-Time Home Buyer Incentive has helped 15,000 families enter the housing market. However, we recognize more needs to be done, particularly in urban centers where supply constraints drive up prices.",
            "speaker": "Minister of Housing",
            "date": "2024-02-20",
            "topic": "housing"
        },
        {
            "text": "Mr. Speaker, small businesses are the backbone of Canada's economy, employing over 8 million Canadians. During the pandemic, our government provided unprecedented support through wage subsidies, rent relief, and interest-free loans. Now, as we recover, we are focusing on helping businesses adapt to digital technologies through the Digital Adoption Program. We have also reduced small business tax rates and eliminated federal taxes on the first $500,000 of business income. These measures support entrepreneurship and job creation across all regions of Canada.",
            "speaker": "Minister of Small Business",
            "date": "2024-02-25",
            "topic": "small_business"
        }
    ]

    print(f"✅ Collected {len(parliament_texts)} parliamentary text segments")
    return parliament_texts

In [ ]:
def create_bonito_dataset(parliament_data):

    print("📊 Creating Bonito-compatible dataset...")

    # Format data for Bonito (similar to the reference example)
    bonito_data = []
    for item in parliament_data:
        bonito_data.append({
            "input": item["text"],  # This will be the context_col
            "speaker": item["speaker"],
            "date": item["date"],
            "topic": item["topic"]
        })

    # Create HuggingFace Dataset
    dataset = Dataset.from_list(bonito_data)

    print(f"✅ Created dataset with {len(dataset)} examples")
    return dataset


In [ ]:
def generate_instruction_dataset():
    print("🤖 Initializing Bonito model...")

    # Initialize Bonito model (as per your reference)
    bonito = Bonito("BatsResearch/bonito-v1")

    # Collect and prepare data
    parliament_data = collect_parliament_data()
    unannotated_dataset = create_bonito_dataset(parliament_data)

    # Select subset for processing (adjust range as needed)
    dataset_subset = unannotated_dataset.select(range(len(unannotated_dataset)))

    print(f"📝 Processing {len(dataset_subset)} parliamentary texts...")

    # Configure sampling parameters (as per your reference)
    sampling_params = SamplingParams(
        max_tokens=256,
        top_p=0.95,
        temperature=0.5,
        n=1
    )

    # Generate synthetic datasets for different task types
    task_types = [
        "summarization",
        "question_answering",
        "sentiment_analysis",
        "classification",
        "information_extraction"
    ]

    all_synthetic_datasets = []

    for task_type in task_types:
        print(f"🔄 Generating {task_type} tasks...")

        try:
            # Generate tasks using Bonito (following your reference pattern)
            synthetic_dataset = bonito.generate_tasks(
                dataset_subset,
                context_col="input",  # Column containing the parliamentary text
                task_type=task_type,
                sampling_params=sampling_params
            )

            # Add task type and metadata to each example
            for i, example in enumerate(synthetic_dataset):
                example["task_type"] = task_type
                example["source_speaker"] = dataset_subset[i]["speaker"]
                example["source_date"] = dataset_subset[i]["date"]
                example["source_topic"] = dataset_subset[i]["topic"]

            all_synthetic_datasets.extend(synthetic_dataset)
            print(f"✅ Generated {len(synthetic_dataset)} {task_type} examples")

            # Small delay to avoid overwhelming the model
            time.sleep(1)

        except Exception as e:
            print(f"⚠️ Error generating {task_type} tasks: {e}")
            continue

    print(f"🎉 Total generated examples: {len(all_synthetic_datasets)}")
    return all_synthetic_datasets


In [ ]:
def add_parliamentary_specific_tasks(base_dataset, parliament_data):
    print("🏛️ Adding parliamentary-specific tasks...")

    parliamentary_tasks = []

    # Custom parliamentary instruction templates
    templates = [
        {
            "instruction": "Analyze the policy position expressed in this parliamentary statement:",
            "task_type": "policy_analysis"
        },
        {
            "instruction": "What are the main arguments presented by this speaker?",
            "task_type": "argument_analysis"
        },
        {
            "instruction": "Identify the political stance (government vs opposition) in this statement:",
            "task_type": "stance_detection"
        },
        {
            "instruction": "Extract specific statistics and figures mentioned in this parliamentary text:",
            "task_type": "data_extraction"
        },
        {
            "instruction": "What policy areas are discussed in this parliamentary statement?",
            "task_type": "topic_classification"
        },
        {
            "instruction": "Summarize the key commitments or promises made in this statement:",
            "task_type": "commitment_extraction"
        }
    ]

    # Generate custom tasks for subset of data
    for i, item in enumerate(parliament_data[:len(parliament_data)//2]):  # Use half for custom tasks
        for template in random.sample(templates, 2):  # 2 tasks per text
            custom_task = {
                "instruction": template["instruction"],
                "input": item["text"],
                "output": generate_custom_output(template["task_type"], item),
                "task_type": template["task_type"],
                "source_speaker": item["speaker"],
                "source_date": item["date"],
                "source_topic": item["topic"]
            }
            parliamentary_tasks.append(custom_task)

    print(f"✅ Added {len(parliamentary_tasks)} parliamentary-specific tasks")
    return base_dataset + parliamentary_tasks


In [ ]:
def generate_custom_output(task_type, item):
    text = item["text"]
    speaker = item["speaker"]
    topic = item["topic"]

    if task_type == "policy_analysis":
        if "Minister" in speaker:
            return f"This statement presents the government's policy position on {topic}, emphasizing achievements and future commitments while defending current approaches."
        else:
            return f"This statement critiques government policy on {topic}, highlighting failures and proposing alternative approaches."

    elif task_type == "argument_analysis":
        return f"The speaker presents arguments focused on {topic}, using statistical evidence and policy examples to support their position while addressing counterarguments."

    elif task_type == "stance_detection":
        if "Minister" in speaker:
            return "Government position: Defending current policies and highlighting achievements."
        else:
            return "Opposition position: Criticizing government policies and proposing alternatives."

    elif task_type == "data_extraction":
        import re
        numbers = re.findall(r'\$?\d+(?:\.\d+)?(?:\s*billion|\s*million|\s*%)?', text)
        return f"Key statistics: {', '.join(numbers[:5]) if numbers else 'Various policy metrics and targets mentioned'}"

    elif task_type == "topic_classification":
        return f"Primary topic: {topic.replace('_', ' ').title()}"

    elif task_type == "commitment_extraction":
        if "Minister" in speaker:
            return f"Key commitments include policy implementation, funding allocation, and performance targets related to {topic}."
        else:
            return f"Proposed commitments include policy reforms, alternative approaches, and improved accountability measures for {topic}."

    return "Analysis of parliamentary statement addressing key policy issues and positions."

In [ ]:
def evaluate_dataset_quality(dataset):

    print("\n📊 Dataset Quality Analysis:")
    print("=" * 50)

    # Basic statistics
    total_examples = len(dataset)

    # Task type distribution
    task_types = {}
    speakers = {}
    topics = {}

    instruction_lengths = []
    output_lengths = []
    unique_instructions = set()

    for item in dataset:
        # Count task types
        task_type = item.get("task_type", "unknown")
        task_types[task_type] = task_types.get(task_type, 0) + 1

        # Count speakers
        speaker = item.get("source_speaker", "unknown")
        speakers[speaker] = speakers.get(speaker, 0) + 1

        # Count topics
        topic = item.get("source_topic", "unknown")
        topics[topic] = topics.get(topic, 0) + 1

        # Length analysis
        instruction_lengths.append(len(item.get("instruction", "")))
        output_lengths.append(len(item.get("output", "")))
        unique_instructions.add(item.get("instruction", ""))

    # Print results
    print(f"📈 Total Examples: {total_examples}")
    print(f"🔤 Unique Instructions: {len(unique_instructions)}")
    print(f"📏 Avg Instruction Length: {sum(instruction_lengths)/len(instruction_lengths):.1f} chars")
    print(f"📏 Avg Output Length: {sum(output_lengths)/len(output_lengths):.1f} chars")

    print(f"\n📋 Task Type Distribution:")
    for task_type, count in sorted(task_types.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / total_examples) * 100
        print(f"  {task_type}: {count} ({percentage:.1f}%)")

    print(f"\n🎤 Speaker Distribution:")
    for speaker, count in sorted(speakers.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / total_examples) * 100
        print(f"  {speaker}: {count} ({percentage:.1f}%)")

    print(f"\n📚 Topic Distribution:")
    for topic, count in sorted(topics.items(), key=lambda x: x[1], reverse=True):
        percentage = (count / total_examples) * 100
        print(f"  {topic}: {count} ({percentage:.1f}%)")

    # Quality assessment
    diversity_score = len(unique_instructions) / total_examples
    balance_score = min(task_types.values()) / max(task_types.values()) if task_types else 0

    print(f"\n🎯 Quality Metrics:")
    print(f"  Diversity Score: {diversity_score:.3f} (higher = more unique instructions)")
    print(f"  Balance Score: {balance_score:.3f} (higher = more balanced task distribution)")

    return {
        "total_examples": total_examples,
        "unique_instructions": len(unique_instructions),
        "task_types": task_types,
        "speakers": speakers,
        "topics": topics,
        "diversity_score": diversity_score,
        "balance_score": balance_score
    }

In [ ]:
def save_dataset(dataset, filename="parliamentary_bonito_dataset.json"):

    print(f"\n💾 Saving dataset...")

    # Save as JSON
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(dataset, f, indent=2, ensure_ascii=False)

    # Save as CSV for easy viewing
    csv_filename = filename.replace('.json', '.csv')
    df = pd.DataFrame(dataset)
    df.to_csv(csv_filename, index=False, encoding='utf-8')

    # Save as HuggingFace dataset
    hf_dataset = Dataset.from_list(dataset)
    hf_dataset_name = filename.replace('.json', '_hf_dataset')
    hf_dataset.save_to_disk(hf_dataset_name)

    # Save sample
    sample_filename = filename.replace('.json', '_sample.json')
    with open(sample_filename, 'w', encoding='utf-8') as f:
        json.dump(dataset[:10], f, indent=2, ensure_ascii=False)

    print(f"✅ Complete dataset: {filename}")
    print(f"✅ CSV format: {csv_filename}")
    print(f"✅ HuggingFace format: {hf_dataset_name}")
    print(f"✅ Sample file: {sample_filename}")


In [ ]:
def improve_diversity_tips():
    print("\n🎯 Tips for Improving Dataset Diversity:")
    print("=" * 50)

    tips = [
        "1. 📊 Data Sources:",
        "   • Collect from multiple parliamentary sessions",
        "   • Include committee hearings, not just main debates",
        "   • Add Question Period exchanges",
        "   • Include private member bills discussions",
        "",
        "2. 🎭 Task Diversity:",
        "   • Add debate outcome prediction",
        "   • Include fact-checking tasks",
        "   • Add rhetoric analysis (persuasive techniques)",
        "   • Include stakeholder impact analysis",
        "",
        "3. 🌈 Content Variation:",
        "   • Balance government vs opposition speakers",
        "   • Include all major political parties",
        "   • Cover federal, provincial, and municipal levels",
        "   • Add different policy complexity levels",
        "",
        "4. 🔧 Technical Improvements:",
        "   • Use different temperature settings for variety",
        "   • Implement quality filtering",
        "   • Add human review for edge cases",
        "   • Use ensemble of different models",
        "",
        "5. 📝 Instruction Engineering:",
        "   • Vary instruction complexity (simple to expert)",
        "   • Use different perspectives (journalist, citizen, analyst)",
        "   • Add multi-step reasoning tasks",
        "   • Include comparative analysis across topics"
    ]

    for tip in tips:
        print(tip)


In [ ]:
def main():
    print("🏛️ Parliamentary Instruction Dataset Creator (Bonito)")
    print("=" * 60)

    try:
        # Step 1: Generate base dataset using Bonito
        synthetic_dataset = generate_instruction_dataset()

        # Step 2: Add parliamentary-specific tasks
        parliament_data = collect_parliament_data()
        enhanced_dataset = add_parliamentary_specific_tasks(synthetic_dataset, parliament_data)

        # Step 3: Evaluate quality
        quality_metrics = evaluate_dataset_quality(enhanced_dataset)

        # Step 4: Save dataset
        save_dataset(enhanced_dataset)

        # Step 5: Show samples
        print(f"\n📋 Sample Examples:")
        print("=" * 30)

        for i, example in enumerate(enhanced_dataset[:3]):
            print(f"\n--- Example {i+1} ---")
            print(f"Task Type: {example['task_type']}")
            print(f"Instruction: {example['instruction']}")
            print(f"Input: {example['input'][:200]}...")
            print(f"Output: {example['output'][:200]}...")
            print(f"Speaker: {example['source_speaker']}")
            print(f"Topic: {example['source_topic']}")

        # Step 6: Improvement suggestions
        improve_diversity_tips()

        print(f"\n🎉 SUCCESS! Generated {len(enhanced_dataset)} instruction examples")
        print("Check the output files to review your dataset.")

    except Exception as e:
        print(f"❌ Error: {e}")
        print("\nTroubleshooting:")
        print("1. Ensure you have installed: pip install bonito-nlp vllm datasets")
        print("2. Check if you have sufficient GPU memory for the model")
        print("3. Try reducing the dataset size if memory issues occur")

if __name__ == "__main__":
    main()

🏛️ Parliamentary Instruction Dataset Creator (Bonito)
🤖 Initializing Bonito model...
INFO 06-08 10:15:46 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-08 10:15:46 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-08 10:15:46 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

WARNING 06-08 10:15:48 [config.py:3096] Your Tesla T4 device (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 06-08 10:15:48 [config.py:3135] Casting torch.bfloat16 to torch.float16.
INFO 06-08 10:16:21 [config.py:793] This model supports multiple tasks: {'embed', 'generate', 'score', 'reward', 'classify'}. Defaulting to 'generate'.
WARNING 06-08 10:16:21 [arg_utils.py:1583] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 06-08 10:16:21 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.0.1) with config: model='BatsResearch/bonito-v1', speculative_config=None, tokenizer='BatsResearch/bonito-v1', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable

tokenizer_config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

...

INFO 06-08 10:16:28 [cuda.py:240] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 06-08 10:16:28 [cuda.py:289] Using XFormers backend.
INFO 06-08 10:16:29 [parallel_state.py:1064] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0, EP rank 0
INFO 06-08 10:16:29 [model_runner.py:1170] Starting to load model BatsResearch/bonito-v1...
INFO 06-08 10:16:30 [weight_utils.py:291] Using model weights format ['*.bin']


pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/5.06G [00:00<?, ?B/s]

INFO 06-08 10:23:13 [weight_utils.py:307] Time spent downloading weights for BatsResearch/bonito-v1: 403.124688 seconds


Loading pt checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
